##  1. Bigram Language Model

In [40]:
with open("wizard_of_oz.txt",'r', encoding='utf-8') as f:
    text = f.read()
print((text[:100]))


  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [41]:
# create a vocab type of list:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [42]:
# encoding gets a character or line to an integer:
# tokenizer tokenizes everything:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# encoding and decoding:
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encode a string for example:
print(encode("hello"))
print(decode(encode("hello")))

[61, 58, 65, 65, 68]
hello


In [43]:
import torch

In [44]:
data = torch.tensor(encode(text), dtype = torch.long) # torch.long is important
print(data[:100])

tensor([ 0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


* The bigram model does exactly this.. given the previous it just predicts the next

In [45]:
# train and validation splits:

n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([0]) target is tensor(1)
when input is tensor([0, 1]) target is tensor(1)
when input is tensor([0, 1, 1]) target is tensor(28)
when input is tensor([ 0,  1,  1, 28]) target is tensor(39)
when input is tensor([ 0,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([ 0,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([ 0,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([ 0,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [46]:
# CPU does tasks sequentially so we can do them in parallel with GPU:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [47]:
block_size = 8
batch_size = 4

In [48]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x,y = get_batch("train")
print('Inputs')
print(x)
print('targets: ')
print(y)

Inputs
tensor([[68, 76,  1, 61, 58,  1, 72, 73],
        [55, 65, 58,  9,  3,  1, 54, 60],
        [ 9,  1,  3, 62, 59,  1, 73, 61],
        [ 1, 54, 72, 64, 58, 57,  1, 50]], device='cuda:0')
targets: 
tensor([[76,  1, 61, 58,  1, 72, 73, 71],
        [65, 58,  9,  3,  1, 54, 60, 71],
        [ 1,  3, 62, 59,  1, 73, 61, 62],
        [54, 72, 64, 58, 57,  1, 50, 58]], device='cuda:0')


In [49]:
import torch.nn as nn
from torch.nn import functional as F

In [50]:
epochs = 5000
learning_rate = 3e-4
eval_epochs = 250
# dropout = 0.2

In [51]:
# so that pytorch uses no gradients to preserve memory and speed:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_epochs)
        for k in range(eval_epochs):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [52]:
class bigrammodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    # nn.embedding is like a lookup table
    def forward(self, index, targets = None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            # logits are like most likely next word coming in(like a probab dist.)
            # B- Batch, T- Time, C- Count
            # shape is used to unpack and view is followed by this to put them back in a tensor
            B, T, C = logits.shape
            # we do this because pytorch expects arguments (N -> B*T, C-> C)
            # we wanted it as (B,C,T) hence we use view ...
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self,index, max_new_tokens):
        # index is (B,T) array of indices in the current context
        
        for _ in range(max_new_tokens):
            # getting predictions
            logits, loss = self.forward(index)
            # focus on the last step of time
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to obtain probabilities:
            # focus on last dimension
            probs = F.softmax(logits, dim = -1) # (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples = 1) #(B,1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim = 1) # does it on (B,T+1)
        return index

In [53]:
# defining the size of vocabSize
vocab_size = len(chars)

In [54]:
model = bigrammodel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


lATn[PDPpjB
xy:o'GZzHVM!I:o0Bm6LZTURN-?9oiAk47IlJaGk4JQSZQs1XPVg(6C!Ht1SZ5Uj'TY)Skv)hKfpLwO'DIaPh9n?Jy7EhyINxXBZQ-oKW _XvzIw2"xwYwT]oBbm83;*XMHg!S!m.Ndd&-OM[W
pjWZtM8qez1q*e&hsQ&TV6snMylM7X)hRoFpWm9_2pyA*BhefvOhTM(bTPaz(sRVk,v0DQ)KW,ZQO'5_5q1x61m9QsEN7BEh;xeZ?oi')ja,kU5e&Fdkvt5[I:'_oE1YOidpsuPglVRNGd)]'hylSKv6_l"W,k,hcj ?je(bqR'gFzNR-P
.&
Fbmdv-QB6Q txa"::iOYhcpsXpWarlV)W,mjyD]8]wbTLHVgX1,HbTfI.8v;)m('t3i
tF)zCuK!ZG(OYUVAZP35]lLXJ0YRPPDhY2Ce&ptaH,4!nx26Nm"fx*(w,c;t6uIYu?&p o3)p-sE0Gy't"DJ3UYU-Md


In [60]:
# create a PyTorch Optimizer:
optim = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(epochs):
    # Loss reporting train vs eval mode:
    if iter % eval_epochs == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.3f}, val loss: {losses["val"]:.3f}')
    # sampling a batch:
    xb, yb = get_batch("train")
    
    # evaluate loss:
    logits, loss = model.forward(xb,yb)
    # instead of 0 set gradient to None
    optim.zero_grad(set_to_none = True)
    loss.backward()
    optim.step()

print(loss.item())

step: 0, train loss: 4.842, val loss: 4.820
step: 250, train loss: 4.742, val loss: 4.788
step: 500, train loss: 4.703, val loss: 4.685
step: 750, train loss: 4.656, val loss: 4.651
step: 1000, train loss: 4.585, val loss: 4.581
step: 1250, train loss: 4.523, val loss: 4.523
step: 1500, train loss: 4.489, val loss: 4.491
step: 1750, train loss: 4.406, val loss: 4.427
step: 2000, train loss: 4.353, val loss: 4.376
step: 2250, train loss: 4.307, val loss: 4.309
step: 2500, train loss: 4.251, val loss: 4.259
step: 2750, train loss: 4.191, val loss: 4.219
step: 3000, train loss: 4.161, val loss: 4.167
step: 3250, train loss: 4.114, val loss: 4.114
step: 3500, train loss: 4.042, val loss: 4.078
step: 3750, train loss: 4.009, val loss: 4.023
step: 4000, train loss: 3.967, val loss: 3.992
step: 4250, train loss: 3.926, val loss: 3.934
step: 4500, train loss: 3.867, val loss: 3.890
step: 4750, train loss: 3.841, val loss: 3.842
3.9494590759277344


In [61]:
model = bigrammodel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


PQ6fVrP uqw5VMIrEw"::[VW-Ts[H.w[qi02A"RukAJMNv[F"h0p;IOhMnBp0Z(XB*3no!"C;8OB-RIhaMn( C)'of,fVIDTIP,*8cK1XI0]"v-]FdBXDAap*?;!TI5IOx_wU'YcFDTd]y D*jyGpA"Ba1nSxgZ9lCfZ :Yk.P,0uHnm7N.o9qk
7"kVci!H6awWAMnps)QzQzk.7y* h0Zw5EF*k8O10,f:V5(Qj]9 f&Uc e;28MRnp;6QZUmXN06O1V!m"2zIOxgfvf06R
!c_fZwolS.7YjD74[lK,43fFAiMrobeqFdeV1o9lQz3*bAZnaKLY(KkGQZe.O
,8ig2"D2sZt40nS.zE
RQumGfwh7_r(n5F*c,fXGQZdNwrfo9t9'?"gavfy_K6Hi_xV_l h7Zk2R,f?RFOkwsArZMgm]h.eBTviYsaF),BIhm7B]5HKFPr1k ?;D bVZU uIIwS:nDi0_(gf1R_t:c3NHnhS(?QX


* Common Optimizers:
1. MSE (Mean Squared Error)- for regression
2. Gradient Descent (GD)- to minimize the loss function
3. Momentum - to smooth out and aid GD, extension of GD
4. RMSprop - Moving average of squared GD
5. Adam - Combines momentum and RMSProp
6. AdamW - modification of Adam by adding weight decay

In [63]:
# Activation functions example ReLU:
x = torch.tensor([-0.05], dtype = torch.float32)
y = nn.ReLU(x)
print(y)

ReLU(inplace=True)


In [64]:
# Activation functions example sigmoid:
x = torch.tensor([-0.05], dtype = torch.float32)
y = F.sigmoid(x)
print(y)

tensor([0.4875])


In [66]:
# Activation functions example tanh:
x = torch.tensor([1], dtype = torch.float32)
y = F.tanh(x)
print(y)

tensor([0.7616])
